<a href="https://colab.research.google.com/github/kim1987/aiffel/blob/main/aiffel/exploration/exlporation_seventeen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

exploration_17
with hugging face  
with https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf

In [ ]:
import torch
import torch.nn as nn
import torchtext

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import pickle
import random
import collections
import json
import requests
from datetime import datetime

In [ ]:
#!pip3 install gitpython

In [ ]:
#import time
#import git
#from git import RemoteProgress

In [ ]:
#import sentencepiece as spm
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
#import seaborn as sns
#from wordcloud import WordCloud

In [ ]:
if torch.cuda.is_available():
  print(torch.cuda.get_device_name())

Tesla V100-SXM2-16GB


데이터 로딩

In [ ]:
'''
os.mkdir('/content/data')
class CloneProgress(RemoteProgress):
  def update(self, op_code, cur_count, max_count=None, message=''):
      if message:
          print(message)

#print('Cloning into %s' % git_root)
git.Repo.clone_from(f'https://github.com/korquad/korquad.github.io', '/content/data', 
        branch='master', progress=CloneProgress())
'''

"\nos.mkdir('/content/data')\nclass CloneProgress(RemoteProgress):\n  def update(self, op_code, cur_count, max_count=None, message=''):\n      if message:\n          print(message)\n\n#print('Cloning into %s' % git_root)\ngit.Repo.clone_from(f'https://github.com/korquad/korquad.github.io', '/content/data', \n        branch='master', progress=CloneProgress())\n"

In [ ]:
'''
class MyDataSet(torch.utils.data.Dataset):

  def __init__(self,root_path, train=True):
    super().__init__()
    self.path = root_path
    self.train = train
      #download
    self.download()
      #unzip
    self.unzip()

  def download(self):
    location = 'train' if self.train else 'dev'
    if not os.path.exists(os.path.join(self.path,location)):
      os.mkdir(os.path.join(self.path,location))
      for i in range(1,14):
        url = f"https://github.com/korquad/korquad.github.io/raw/master/dataset/KorQuAD_2.0/train/KorQuAD_2.0_train_{str(i).zfill(2)}.zip"

        response = requests.get(url, stream=True)
        total_size_in_bytes= int(response.headers.get('content-length', 0))
        chunk_size = 4*1024
        progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True,desc = f'{i}\'th file')
        with open(os.path.join(self.path,location,f'train{str(i).zfill(2)}.zip'), 'wb') as file:
            for data in response.iter_content(chunk_size=chunk_size):
                progress_bar.update(len(data))
                file.write(data)
        progress_bar.close()

  def unzip(self):
    location = 'train' if self.train else 'dev'
    if not os.path.exists(os.path.join(self.path,location,'unzip')):
      os.mkdir(os.path.join(self.path,location,'unzip'))
      for i in range(1,14):
        torchtext.utils.extract_archive(os.path.join(self.path,location,f'train{str(i).zfill(2)}.zip'),os.path.join(self.path,location,'unzip'))
'''

'\nclass MyDataSet(torch.utils.data.Dataset):\n\n  def __init__(self,root_path, train=True):\n    super().__init__()\n    self.path = root_path\n    self.train = train\n      #download\n    self.download()\n      #unzip\n    self.unzip()\n\n  def download(self):\n    location = \'train\' if self.train else \'dev\'\n    if not os.path.exists(os.path.join(self.path,location)):\n      os.mkdir(os.path.join(self.path,location))\n      for i in range(1,14):\n        url = f"https://github.com/korquad/korquad.github.io/raw/master/dataset/KorQuAD_2.0/train/KorQuAD_2.0_train_{str(i).zfill(2)}.zip"\n\n        response = requests.get(url, stream=True)\n        total_size_in_bytes= int(response.headers.get(\'content-length\', 0))\n        chunk_size = 4*1024\n        progress_bar = tqdm(total=total_size_in_bytes, unit=\'iB\', unit_scale=True,desc = f\'{i}\'th file\')\n        with open(os.path.join(self.path,location,f\'train{str(i).zfill(2)}.zip\'), \'wb\') as file:\n            for data in resp

---------giveup--------

json from aiffel


In [ ]:
!cp /content/drive/MyDrive/colabdata/aiffel/ex_seventeen/KorQuAD_v1.0_dev.json /content
!cp /content/drive/MyDrive/colabdata/aiffel/ex_seventeen/KorQuAD_v1.0_train.json /content

In [ ]:
data_dev = pd.read_json('/content/KorQuAD_v1.0_dev.json',lines=True)

In [ ]:
data_train = pd.read_json('/content/KorQuAD_v1.0_train.json',lines=True)

In [ ]:
data_dev = data_dev['data']
data_train = data_train['data']

In [ ]:
len(data_train[0])

1420

In [ ]:
def split_data(data):
  pd_train = []
  for text in tqdm(data[0]):
    for content in text['paragraphs']:
      context = content['context']
      for question in content['qas']:
        ans = question['answers'][0]['text']
        qas = question['question']
        pd_train.append({'context':context+'[SEP]'+qas,'ans':ans})
  return pd.DataFrame(pd_train)

In [ ]:
pd_train = split_data(data_train)

  0%|          | 0/1420 [00:00<?, ?it/s]

In [ ]:
pd_dev = split_data(data_dev)

  0%|          | 0/140 [00:00<?, ?it/s]

In [ ]:
!pip install transformers
!pip install SentencePiece
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-jq_6jsss/kobert-tokenizer_fc5f1e4573ba4d93ac7ffe5f6883a153
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-jq_6jsss/kobert-tokenizer_fc5f1e4573ba4d93ac7ffe5f6883a153


In [ ]:
import transformers

In [ ]:
import kobert_tokenizer

In [ ]:
tokenizer = kobert_tokenizer.KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
max_len = 498
pd_train=pd_train[pd_train['context'].apply(len)<=max_len].reset_index().drop('index',axis=1)
pd_dev=pd_dev[pd_dev['context'].apply(len)<=max_len].reset_index().drop('index',axis=1)

In [ ]:
tqdm.pandas()

In [ ]:
pd_train['context'] = pd_train['context'].progress_apply(lambda x : tokenizer.encode_plus(x,padding='max_length',max_length=max_len+2))
pd_train['ans'] = pd_train['ans'].progress_apply(lambda x : tokenizer.encode_plus(x)['input_ids'])
pd_dev['context'] = pd_dev['context'].progress_apply(lambda x : tokenizer.encode_plus(x,padding='max_length',max_length=max_len+2))
pd_dev['ans'] = pd_dev['ans'].progress_apply(lambda x : tokenizer.encode_plus(x)['input_ids'])

  0%|          | 0/29683 [00:00<?, ?it/s]

  0%|          | 0/29683 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

In [ ]:
def find(data):
  data['ansloc']=[[max_len+1,max_len+1]]*len(data)
  for idx in tqdm(range(len(data))):
    len_ans = len(data['ans'][idx])-2
    ans = data['ans'][idx][1:-1]
    look_up = data['context'][idx]['input_ids']
    for i,loc in enumerate(look_up):
      if loc == ans[0] and ans == look_up[i:i+len_ans]:
        data['ansloc'][idx]=[i,i+len_ans]

In [ ]:
find(pd_train)
find(pd_dev)

  0%|          | 0/29683 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

In [ ]:
pd_train = pd_train[pd_train['ansloc'].apply(lambda x : x!=[499,499])].reset_index().drop('index',axis=1)
pd_dev = pd_dev[pd_dev['ansloc'].apply(lambda x : x!=[499,499])].reset_index().drop('index',axis=1)

In [ ]:
class MydataSet(torch.utils.data.Dataset):

  def __init__(self,data):
    super().__init__()
    self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    return (torch.tensor(self.data['context'][idx]['input_ids']),torch.tensor(self.data['context'][idx]['attention_mask']),torch.tensor(self.data['context'][idx]['token_type_ids'])),\
    (torch.cat([torch.tensor(self.data['ans'][idx]),torch.ones([500-len(self.data['ans'][idx])],dtype=torch.int32)]),torch.tensor(self.data['ansloc'][idx]))

In [ ]:
batch_size=16
set_train = MydataSet(pd_train)
set_dev = MydataSet(pd_dev)
load_train = torch.utils.data.DataLoader(set_train,batch_size=batch_size,shuffle=True)
load_dev = torch.utils.data.DataLoader(set_dev,batch_size=batch_size)

In [ ]:
next(iter(load_train))[1][1].transpose(0,1)

tensor([[ 42,   1,   4,  30,  32, 159, 160,  50,  53, 101, 120,  43, 148, 245,
         170,  68],
        [ 44,   6,   5,  37,  37, 162, 163,  56,  57, 105, 123,  49, 150, 246,
         173,  70]])

[SEP] 으로 나눔 값은 3.

모델

In [ ]:
import gc
gc.collect()

90

In [ ]:
#torch.cuda.ipc_collect()
#torch.cuda.empty_cache()

In [ ]:
#torch.cuda.memory_summary()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [ ]:
class BERT4KorQuAD(nn.Module):
    def __init__(self):
        super().__init__()

        self.bert = transformers.BertModel.from_pretrained('skt/kobert-base-v1')
        self.linear = nn.Linear(self.bert.config.hidden_size , 2)
    
    def forward(self, input_ids,attention_mask,token_type_ids):
        logits_lm = self.bert(input_ids,attention_mask,token_type_ids)

        hidden = self.linear(logits_lm[0]) # (bs, n_seq, 2)
        start_logits, end_logits = torch.chunk(hidden,2,2)  # (bs, n_seq, 1), (bs, n_seq, 1)

        start_logits = torch.squeeze(start_logits, axis=-1)
        start_outputs = torch.nn.Softmax(dim=1)(start_logits)

        end_logits = torch.squeeze(end_logits, axis=-1)
        end_outputs = torch.nn.Softmax(dim=1)(end_logits)

        return start_outputs, end_outputs

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Tesla V100-SXM2-16GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [ ]:
model = BERT4KorQuAD()
model.to(device)

BERT4KorQuAD(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [ ]:
epoch = 40

In [ ]:
loss_graph = []
accuracy_graph = []

In [ ]:
loss_start = nn.CrossEntropyLoss()
loss_end = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

In [ ]:
for i in range(epoch):
  model.train()
  train_data = load_train
  for j,(src , tgt) in enumerate(tqdm(train_data)):

    optimizer.zero_grad()

    pred_start, pred_end = model(src[0].to(device),src[1].to(device),src[2].to(device))

    
    tgt = tgt[1].transpose(0,1)
    loss = loss_start(pred_start,tgt[0].to(device)) + loss_end(pred_end,tgt[1].to(device))

    loss.backward()

    optimizer.step()

    if j%100 == 0:
      match =(pred_start.argmax(1)==tgt[0].to(device)).sum()+(pred_end.argmax(1)==tgt[1].to(device)).sum()
      total = 2*tgt[0].size(0)
      print('\rloss: ',loss.item(),'   accuracy: ',match.item()/total,end="",flush=True)
      loss_graph.append(loss.item())
      accuracy_graph.append(match.item()/total)
    
  print('\nepoch:',i,flush=True)

  0%|          | 0/1507 [00:00<?, ?it/s]

loss:  12.42923355102539    accuracy:  0.0
epoch: 0


  0%|          | 0/1507 [00:00<?, ?it/s]

loss:  12.429254531860352    accuracy:  0.0
epoch: 1


  0%|          | 0/1507 [00:00<?, ?it/s]

...

  0%|          | 0/1507 [00:00<?, ?it/s]

loss:  12.429281234741211    accuracy:  0.0
epoch: 35


  0%|          | 0/1507 [00:00<?, ?it/s]

loss:  12.429154396057129    accuracy:  0.0

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/models/bert_weights123.pth')

In [ ]:
model.eval()
dev_data = load_dev
match =0
total =0
with torch.no_grad():
  for src , tgt in tqdm(dev_data):


    pred_start, pred_end = model(src[0].to(device),src[1].to(device),src[2].to(device))

    tgt = tgt[1].transpose(0,1)
    match += torch.sum(pred_start.argmax(1)==tgt[0].to(device))
    match += torch.sum(pred_end.argmax(1)==tgt[1].to(device)) #(pred_end.argmax(1)==tgt[1].to(device)).sum().item()
    total+=2*batch_size # 2* labels.size(0)

  print('accuracy: ',match/total)

  0%|          | 0/151 [00:00<?, ?it/s]

accuracy:  tensor(0.0043, device='cuda:0')
